# Factor Portfolios

Goal: To identify a statistical driver that predicts returns and go long the stocks with highest correlation

#### Momentum Strategy: A tendency of securities to continue moving in the same direction as they have in the past

A statistical factor that measures the trend. Long the stocks that are exhibiting highest momentum and short the lowest momentum. Identify the herd behavior and long / short the stocks

In [ ]:
import pandas as pd
import numpy as np
import datetime
import threading

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *

from openbb_terminal.sdk import openbb

In [ ]:
lastBusDay = datetime.datetime.today()
wk_day = datetime.date.weekday(lastBusDay)
if wk_day > 4:      #if it's Saturday or Sunday
    lastBusDay = lastBusDay - datetime.timedelta(days = wk_day-4) 

today = lastBusDay.strftime("%Y-%m-%d")
top_N = 5

In [ ]:
today = '2023-12-22'

In [ ]:
# today = datetime.datetime.now() - datetime.timedelta(1)
# today = today.strftime("%Y-%m-%d")

In [ ]:
gold_cross_above = openbb.stocks.screener.screener_data('goldcross_usa_mid_gt')

In [ ]:
gold_cross_below = openbb.stocks.screener.screener_data(preset_loaded= 'goldcross_usa_mid_lt')

## Data Preparation

In [ ]:
tickers = gold_cross_above.Ticker.tolist()
tickers += gold_cross_below.Ticker.tolist()
len(tickers)

In [ ]:
stocks = []
for t in tickers:
    df = (
        openbb.stocks.load(t, start_date='2015-01-01', end_date=today, verbose=False)
    )
    df['ticker'] = t
    stocks.append(df)

In [ ]:
stock_historic = pd.concat(stocks)

In [ ]:
stock_prices = pd.concat(stocks)

In [ ]:
stock_prices

In [ ]:
stock_prices.drop(['Close','Dividends','Stock Splits'], axis=1, inplace=True)

In [ ]:
stock_prices.columns = ['open','high','low','close','volume','ticker']

## Factor Engineering

In [ ]:
# check to make sure we have atleast 2 years of data
days_of_data = stock_prices.groupby('ticker').size()

In [ ]:
mask = days_of_data[days_of_data > 2 * 12 * 21].index # gets index of rows that have count > 2 years * 12 months * 21 days per month
stock_prices = stock_prices[stock_prices.ticker.isin(mask)] #filter out portfolios only that match the criteria

In [ ]:
#data munging
stock_prices = (
    stock_prices
    .set_index('ticker', append=True)
    .reorder_levels(['ticker','date'])
).drop_duplicates()

In [ ]:
def momentum(close):
    returns = close.pct_change()[-126:] # get returns for previous 126 days
    return (
        (close[-21] - close[-252])/close[-252] # monthly returns per year
        - (close[-1] - close[-21]) / close[-21] # daily returns per month
    ) / np.std(returns) # normalized using standard deviation

In [ ]:
df = (
    stock_prices
    .groupby(
        'ticker',
        group_keys=False
    )
    .rolling(252)
    .close
    .apply(momentum)
)

In [ ]:
#pandas adds an extra ticker index, drop it
df.index = df.index.droplevel(0)

In [ ]:
stock_prices['momentum'] = df
stock_prices.dropna(inplace=True)

In [ ]:
# rank the momentum
stock_prices['factor_rank'] = (
    stock_prices
    .groupby(level=[1])
    .momentum
    .rank(ascending=False)
)

In [ ]:
# stocks_to_buy = (
#     stock_prices
#     .xs(today, level=1)
#     .sort_values('factor_rank')
#     .head(10)
# )

In [ ]:
start_date = datetime.date(2023,12,22)
selected_stocks = []
for i in range(0, 5):
    ix = (
    stock_prices
    .xs((start_date - datetime.timedelta(i)).strftime('%Y-%m-%d'), level=1)
    .sort_values('factor_rank')
    .head(10)
).index.tolist()
    selected_stocks.append(ix)

In [ ]:
#get unique stocks from the past 5 days that exhibit momentum
from functools import reduce
ticks = list(reduce(lambda i, j: i|j, (set(x) for x in selected_stocks)))
ticks

# Risk Parity

In [ ]:
import riskfolio as rp

In [ ]:
def calc_returns(df):
    df['daily_returns'] = df['close'].pct_change()
    return df

In [ ]:
# filter for stock prices that are in selected ticks
df = stock_prices[stock_prices.index.get_level_values('ticker').isin(ticks)]

In [ ]:
data = (df.groupby(level = 'ticker',group_keys=False).apply(calc_returns))

In [ ]:
data = data.unstack(level=0)

In [ ]:
data.dropna(inplace=True)

In [ ]:
portfolio = rp.Portfolio(returns=data.daily_returns)

In [ ]:
# Define params for risk parity optimizer
portfolio.lowerret = 0.0008
portfolio.assets_stats(method_mu="hist", method_cov="hist", d=0.94)
w_rp = portfolio.rp_optimization(model="Classic", rm="MV", rf=0.05, hist=True)

In [ ]:
investment_amount = 100_000
w_rp['investible_amount'] = investment_amount * w_rp

In [ ]:
w_rp.investible_amount.sum()

In [ ]:
w_rp['last_price'] = data.close.iloc[-1]
w_rp['shares'] = 0

In [ ]:
def allocate(df, amount):
    no_more_allocated = True
    for row in df.itertuples():
        last_price = w_rp.at[row.Index, 'last_price']
        invested_amount = w_rp.at[row.Index, 'shares'] * last_price 
        if last_price < amount and w_rp.at[row.Index, 'investible_amount'] > (invested_amount+last_price):
            amount -= last_price
            w_rp.at[row.Index, 'shares'] += 1
            no_more_allocated = False
        else:
            continue
    return amount, no_more_allocated

In [ ]:
investment_amount

In [ ]:
remaining = investment_amount
stopTheLoop = False
while(True):
    if remaining > 100 and stopTheLoop == False:
        remaining, stopTheLoop = allocate(w_rp, remaining)
    else:
        break
    

In [ ]:
w_rp

# We now have the model portfolio, rebalance account based on this

In [ ]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *

In [ ]:
class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.pos_df = pd.DataFrame(columns=['Account', 'Symbol', 'SecType',
                                            'Currency', 'Position', 'Avg cost']) 
    
    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextOrderId = orderId
    
    def position(self, account, contract, position, avgCost):
        super().position(account, contract, position, avgCost)
        
        dictionary = {"Account":account, "Symbol": contract.symbol, "SecType": contract.secType,
                        "Currency": contract.currency, "Position": position, "Avg cost": avgCost}
        if self.pos_df["Symbol"].str.contains(contract.symbol).any():
            self.pos_df.loc[self.pos_df["Symbol"]==contract.symbol,"Position"] = position
            self.pos_df.loc[self.pos_df["Symbol"]==contract.symbol,"Avg cost"] = avgCost
        else:
            self.pos_df = pd.concat([self.pos_df,pd.DataFrame(dictionary, index=[0])], ignore_index=True)

In [ ]:
import time
def stock_contract(symbol, secType="STK", exchange="SMART", currency="USD"):
        contract = Contract()
        contract.symbol = symbol
        contract.secType = secType
        contract.exchange = exchange
        contract.currency = currency
        return contract
    
def submit_order(contract, direction, qty=100, orderType="MKT", transmit=True):
    order = Order()
    order.action = direction
    order.totalQuantity = qty
    order.orderType = orderType
    order.transmit = transmit
    order.eTradeOnly = ""
    order.firmQuoteOnly = ""
    # submit order
    app.placeOrder(app.nextOrderId, contract, order)
    app.nextOrderId += 1

In [ ]:
def run_loop():
    app.run()

app = IBapi()
app.connect('127.0.0.1', 7497, 1) # verify this on ibroker client
app.nextOrderId = None

api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

counter = 1
while True:
    if isinstance(app.nextOrderId, int):
        print("Connected")
        break
    else:
        counter = counter+1
        print(f"Waiting {counter}")
        if counter == 60:
            app.disconnect()
            break
        time.sleep(1)

In [ ]:
app.reqPositions()
time.sleep(1)
pos_df = app.pos_df

In [ ]:
w_rp['shares'] = w_rp['shares'].astype(float)
pos_df['Position'] = pos_df['Position'].astype(float)

In [ ]:
df_change = (pd.merge(w_rp, pos_df.set_index('Symbol'),left_index=True,right_index=True, how="outer")
    .fillna(0.0)
    .assign(change = lambda x: x.shares - x.Position)
    .assign(buy = lambda x: np.where(x.change > 0, x.change, 0))
    .assign(sell = lambda x: np.where(x.change < 0, abs(x.change), 0))
)

In [ ]:
df_buy = df_change[df_change.buy > 0]
df_sell = df_change[df_change.sell > 0]

In [ ]:
for row in df_sell.itertuples():
    contract = stock_contract(row.Index)
    qty = row.sell
    # if qty != 0:
        # submit_order(contract, direction="SELL", qty=qty)

In [ ]:
for row in df_buy.itertuples():
    contract = stock_contract(row.Index)
    qty = row.buy
    if qty != 0:
        submit_order(contract, direction="BUY", qty=row.buy)

In [500]:
app.disconnect()